In [47]:
import requests
import pandas as pd

In [ ]:
# https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3
# https://data.sfgov.org/Public-Safety/Police-Department-Calls-for-Service/hz9m-tj6z
# https://data.sfgov.org/City-Infrastructure/Street-Tree-List/tkzw-k3nq
# https://data.sfgov.org/Housing-and-Buildings/Eviction-Notices/5cei-gny5
# https://data.sfgov.org/City-Infrastructure/Historical-Street-Sweeper-Scheduled-Routes/u2ac-gv9v

In [2]:
data = []
for page in range(1, 10):
    limit = 2000
    offset = page * limit
    query = f"$limit={limit}&$offset={offset}" \
        "&$order=updated_datetime DESC" \
        "&$where=updated_datetime between '2019-01-22' and '2019-01-23'"
    sf = f'https://data.sfgov.org/resource/ktji-gk7t.json?{query}'
    url = sf
    r = requests.get(url)
    if r.status_code != 200:
        print(r)
    _next_data = r.json()
    data.extend(_next_data)
    if len(_next_data) < limit:
        break
    
columns = (
    'address', 
    'agency_responsible', 
    'closed_date',
    'lat',
    'long',
    'neighborhoods_sffind_boundaries',
    'point',
    'police_district',
    'requested_datetime',
    'service_details',
    'service_name',
    'service_request_id',
    'service_subtype',
    'source',
    'status_description',
    'status_notes',
    'supervisor_district',
    'updated_datetime',
)
df = pd.DataFrame(data, columns=columns)

In [58]:
to_map = df[(df.supervisor_district == '6') & (df.service_name == 'Encampments')]
to_map = to_map.reset_index()

In [66]:
import folium

from folium.plugins import MarkerCluster


m = folium.Map(zoom_start=13, control_scale=True) #, width=300, height=200)

cluster = MarkerCluster().add_to(m)

for index, row in to_map.iterrows():
    lat = float(row.lat)
    lng = float(row.long)
    centroid = [lat, lng]
    icon = folium.Icon(color='blue')
    popup = row.service_name + row.supervisor_district
    marker = folium.Marker(location=centroid, popup=popup, icon=icon)
    #marker.add_to(cluster)
    marker.add_to(m)
    
m.fit_bounds(m.get_bounds())
m

In [11]:
pd.concat([df.supervisor_district]).value_counts()

9     267
6     256
10    232
8     220
5     185
3     179
11    136
7     108
1      82
2      76
4      49
Name: supervisor_district, dtype: int64

In [48]:
pd.concat([df.agency_responsible]).value_counts()[:50]

DPW Ops Queue                                        672
DPT Abandoned Vehicles Work Queue                    212
Recology_Abandoned                                   105
HSOC Queue                                           100
DPW BSM Queue                                         69
Recology_Overflowing                                  67
Parking Enforcement Dispatch Queue                    53
Duplicate Case Hold Queue                             47
MUNI Work Queue                                       36
PUC Sewer Ops                                         35
DPT Meter_Bike Queue                                  30
SFMTA - Parking Enforcement - G                       23
DPH - Environmental Health - G                        23
311 Supervisor Queue                                  21
DBI Inspection Services Queue                         21
DPW IDC Queue                                         20
DPW BSSR Queue                                        20
DPT - Traffic Signal - G       

In [37]:
pd.concat([df.service_name]).value_counts()

Street and Sidewalk Cleaning                             535
Abandoned Vehicle                                        210
Graffiti                                                 197
Encampments                                              143
Tree Maintenance                                         116
Street Defects                                            71
General Request - PUBLIC WORKS                            69
Parking Enforcement                                       64
Damaged Property                                          54
MUNI Feedback                                             49
General Request - MTA                                     39
Sidewalk or Curb                                          38
Sewer Issues                                              35
Sign Repair                                               29
Rec and Park Requests                                     26
Streetlights                                              24
Blocked Street or SideWa

In [19]:
pd.concat([df.police_district]).value_counts()

MISSION       331
INGLESIDE     291
NORTHERN      199
SOUTHERN      185
BAYVIEW       174
CENTRAL       166
TARAVAL       154
PARK          124
RICHMOND       97
TENDERLOIN     69
Name: police_district, dtype: int64